In [1]:
import json
import requests
import numpy as np
from dateutil.relativedelta import relativedelta
from config import lw_key
import pandas as pd
from datetime import date, timedelta, datetime
from sklearn.svm import SVC 
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


In [2]:
url= "https://api.worldweatheronline.com/premium/v1/weather.ashx?"
past_url = "https://api.worldweatheronline.com/premium/v1/past-weather.ashx?"
# query_url =f"{url}appid={api_key}&zip="

In [3]:
zipcode = input("Enter 5-digit Zipcode:")

Enter 5-digit Zipcode:21061


In [4]:
# zip_df = nomi.query_postal_code(zipcode)


In [5]:
# pandas apply pgeocode to ZIPCODE1
# md_zips = df['ZIPCODE1'].apply(nomi.query_postal_code)
# md_zips.head()

In [6]:
response = requests.get(f"{url}key={lw_key}&q={zipcode}&num_of_days=8&tp=24&cc=no&mca=no&aqi=yes&format=json").json()
response = response['data']


# Forecast Data

In [7]:
weather_dict ={
    "Dates":[],
    "Cloudcover":[],
    "Humidity":[],
    "PrecipInch":[],
    "Pressure":[],
    "FeelsLike":[],
    "HeatIndex":[],
    "MaxTemp":[],
    "MinTemp":[],
    "SunHours":[],
    "UVIndex":[],
}

weather_dict["Dates"]= ([response["weather"][i]["date"] for i in range(len(response["weather"]))])
weather_dict["Cloudcover"]=([response["weather"][i]["hourly"][0]["cloudcover"] for i in range(len(response["weather"]))])
weather_dict["Humidity"]= ([response["weather"][i]["hourly"][0]["humidity"] for i in range(len(response["weather"]))])
weather_dict["PrecipInch"]= ([response["weather"][i]["hourly"][0]["precipInches"] for i in range(len(response["weather"]))])
weather_dict["Pressure"]= ([response["weather"][i]["hourly"][0]["pressure"] for i in range(len(response["weather"]))])
weather_dict["FeelsLike"]= ([response["weather"][i]["hourly"][0]["FeelsLikeF"] for i in range(len(response["weather"]))])
weather_dict["HeatIndex"]= ([response["weather"][i]["hourly"][0]["HeatIndexF"] for i in range(len(response["weather"]))])
weather_dict["MaxTemp"]= ([response["weather"][i]["maxtempF"] for i in range(len(response["weather"]))])
weather_dict["MinTemp"]=([response["weather"][i]["mintempF"] for i in range(len(response["weather"]))])
weather_dict["SunHours"]=([response["weather"][i]["sunHour"] for i in range(len(response["weather"]))])
weather_dict["UVIndex"]= ([response["weather"][i]["hourly"][0]["uvIndex"] for i in range(len(response["weather"]))])
weather_df= pd.DataFrame.from_dict(weather_dict,orient='index').transpose()
weather_df= weather_df.apply(pd.to_numeric, errors= 'ignore')
weather_df["TempDelta"] = weather_df.MaxTemp - weather_df.MinTemp
weather_df["BarChange"] =weather_df["Pressure"].pct_change()
weather_df["HeatChange"] =weather_df["HeatIndex"].pct_change()
weather_df["HumChange"] =weather_df["Humidity"].pct_change()
weather_df= weather_df.iloc[1:]
weather_df.head(10)


,Dates,Cloudcover,Humidity,PrecipInch,Pressure,FeelsLike,HeatIndex,MaxTemp,MinTemp,SunHours,UVIndex,TempDelta,BarChange,HeatChange,HumChange
1,2021-04-17,73,63,0.0,1011,45,49,53,46,6.6,3,7,0.001982,0.020833,0.032787
2,2021-04-18,48,61,0.0,1013,49,51,56,44,10.4,4,12,0.001978,0.040816,-0.031746
3,2021-04-19,68,71,0.0,1015,51,51,58,47,6.6,4,11,0.001974,0.000000,0.163934
4,2021-04-20,27,70,0.0,1017,55,56,64,49,10.4,1,15,0.001970,0.098039,-0.014085
5,2021-04-21,35,60,0.0,1012,46,50,55,43,10.4,3,12,-0.004916,-0.107143,-0.142857
6,2021-04-22,15,50,0.0,1021,42,47,57,38,11.6,3,19,0.008893,-0.060000,-0.166667
7,2021-04-23,16,49,0.0,1023,53,55,63,46,11.6,4,17,0.001959,0.170213,-0.020000


# Historical Data

In [8]:
dates= input("Dates you had a migraine (YYYY-MM-DD):")

Dates you had a migraine (YYYY-MM-DD):1-08,2020-11-09,2020-11-10,2020-11-11,2020-11-12,2020-11-13,2020-11-14,2020-11-17,2020-11-18,2020-11-19,2020-11-20,2020-11-21,2020-11-22,2020-11-25,2020-11-26,2020-11-28,2020-11-29,2020-11-30,2020-12-04,2020-12-05,2020-12-06,2020-12-07,2020-12-09,2020-12-11,2020-12-12,2020-12-13,2020-12-14,2020-12-15,2020-12-16,2020-12-17,2020-12-23,2020-12-24,2020-12-26,2020-12-29,2020-12-30,2021-01-03,2021-01-04,2021-01-06,2021-01-07,2021-01-09,2021-01-10,2021-01-12 ,2021-01-15,2021-01-17,2021-01-18,2021-01-20,2021-01-23,2021-01-26,2021-01-27,2021-01-31,2021-02-02,2021-02-06,2021-02-08,2021-02-09,2021-02-10,2021-02-11,2021-02-15,2021-02-16,2021-02-20,2021-02-21,2021-02-24,2021-02-27,2021-03-01,2021-03-05,2021-03-08


In [9]:
dates=dates.split(",")
dates= [i.strip() for i in dates]
dates_df = pd.DataFrame(dates,columns=['Dates'])
dates_df["Migraine"] = 1


In [10]:
history_dict ={
    "Dates":[],
    "Cloudcover":[],
    "Humidity":[],
    "PrecipInch":[],
    "Pressure":[],
    "FeelsLike":[],
    "HeatIndex":[],
    "MaxTemp":[],
    "MinTemp":[],
    "SunHours":[],
    "UVIndex":[],
    }

In [11]:
for i in range(6,0,-1):
    date= (datetime.today()-relativedelta(months=i)).strftime('%Y-%m-%d')
    enddate=(datetime.today()-relativedelta(months=i-1)).strftime('%Y-%m-%d')
    responses = requests.get(f"{past_url}key={lw_key}&q={zipcode}&date={date}&enddate={enddate}&tp=24&mca=no&aqi=yes&format=json").json()
    responses = responses['data']
    history_dict["Dates"].append([responses["weather"][i]["date"] for i in range(len(responses["weather"]))])
    history_dict["Cloudcover"].append([responses["weather"][i]["hourly"][0]["cloudcover"] for i in range(len(responses["weather"]))])
    history_dict["Humidity"].append([responses["weather"][i]["hourly"][0]["humidity"] for i in range(len(responses["weather"]))])
    history_dict["PrecipInch"].append([responses["weather"][i]["hourly"][0]["precipInches"] for i in range(len(responses["weather"]))])
    history_dict["Pressure"].append([responses["weather"][i]["hourly"][0]["pressure"] for i in range(len(responses["weather"]))])
    history_dict["FeelsLike"].append([responses["weather"][i]["hourly"][0]["FeelsLikeF"] for i in range(len(responses["weather"]))])
    history_dict["HeatIndex"].append([responses["weather"][i]["hourly"][0]["HeatIndexF"] for i in range(len(responses["weather"]))])
    history_dict["MaxTemp"].append([responses["weather"][i]["maxtempF"] for i in range(len(responses["weather"]))])
    history_dict["MinTemp"].append([responses["weather"][i]["mintempF"] for i in range(len(responses["weather"]))])
    history_dict["SunHours"].append([responses["weather"][i]["sunHour"] for i in range(len(responses["weather"]))])
    history_dict["UVIndex"].append([responses["weather"][i]["hourly"][0]["uvIndex"] for i in range(len(responses["weather"]))])

In [12]:
history_df= pd.DataFrame.from_dict(history_dict,orient='index').transpose()
history_df= history_df.apply(pd.to_numeric, errors= 'ignore')


In [13]:
history_df= history_df.apply(pd.Series.explode).reset_index(drop=True)
history_df= history_df.apply(pd.to_numeric, errors= 'ignore')
history_df.reset_index(drop=True)
history_df.head()

,Dates,Cloudcover,Humidity,PrecipInch,Pressure,FeelsLike,HeatIndex,MaxTemp,MinTemp,SunHours,UVIndex
0,2020-10-16,76,60,0.0,1022,56,57,58,54,7.1,3
1,2020-10-17,8,51,0.0,1026,50,53,62,39,11.1,4
2,2020-10-18,1,54,0.0,1029,58,58,63,54,11.0,5
3,2020-10-19,32,78,0.0,1026,61,61,67,55,11.0,5
4,2020-10-20,21,76,0.0,1024,65,66,75,58,11.0,5


In [14]:
history_df["TempDelta"]= history_df.MaxTemp - weather_df.MinTemp
history_df["BarChange"]= history_df["Pressure"].pct_change()
history_df["HeatChange"]= history_df["HeatIndex"].pct_change()
history_df["HumChange"]= history_df["Humidity"].pct_change()
history_df= pd.merge(history_df,dates_df, on = 'Dates', how = 'left')
history_df= history_df.fillna(0)
history_df= history_df.iloc[1:]
history_df.head(10)

,Dates,Cloudcover,Humidity,PrecipInch,Pressure,FeelsLike,HeatIndex,MaxTemp,MinTemp,SunHours,UVIndex,TempDelta,BarChange,HeatChange,HumChange,Migraine
1,2020-10-17,8,51,0.0,1026,50,53,62,39,11.1,4,16.0,0.003914,-0.070175,-0.150000,0.0
2,2020-10-18,1,54,0.0,1029,58,58,63,54,11.0,5,19.0,0.002924,0.094340,0.058824,0.0
3,2020-10-19,32,78,0.0,1026,61,61,67,55,11.0,5,20.0,-0.002915,0.051724,0.444444,0.0
4,2020-10-20,21,76,0.0,1024,65,66,75,58,11.0,5,26.0,-0.001949,0.081967,-0.025641,0.0
5,2020-10-21,57,84,0.0,1024,72,72,77,66,7.0,5,34.0,0.000000,0.090909,0.105263,0.0
6,2020-10-22,7,79,0.0,1023,74,74,80,66,10.9,6,42.0,-0.000977,0.027778,-0.059524,0.0
7,2020-10-23,20,79,0.0,1021,68,70,73,64,10.9,6,27.0,-0.001955,-0.054054,0.000000,0.0
8,2020-10-24,40,74,0.1,1017,68,69,76,62,8.5,5,0.0,-0.003918,-0.014286,-0.063291,0.0
9,2020-10-25,99,75,0.7,1021,47,50,50,46,5.4,2,0.0,0.003933,-0.275362,0.013514,0.0
10,2020-10-26,88,86,0.1,1021,56,57,65,39,5.4,4,0.0,0.000000,0.140000,0.146667,0.0


# Machine Learning Pre-Processing of Data

In [15]:
hist_ml_df=history_df.drop(columns =["Dates"])
hist_ml_df

,Cloudcover,Humidity,PrecipInch,Pressure,FeelsLike,HeatIndex,MaxTemp,MinTemp,SunHours,UVIndex,TempDelta,BarChange,HeatChange,HumChange,Migraine
1,8,51,0.0,1026,50,53,62,39,11.1,4,16.0,0.003914,-0.070175,-0.150000,0.0
2,1,54,0.0,1029,58,58,63,54,11.0,5,19.0,0.002924,0.094340,0.058824,0.0
3,32,78,0.0,1026,61,61,67,55,11.0,5,20.0,-0.002915,0.051724,0.444444,0.0
4,21,76,0.0,1024,65,66,75,58,11.0,5,26.0,-0.001949,0.081967,-0.025641,0.0
5,57,84,0.0,1024,72,72,77,66,7.0,5,34.0,0.000000,0.090909,0.105263,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,65,78,0.1,1007,58,59,64,53,7.8,4,0.0,0.005994,-0.092308,-0.060241,0.0
184,69,68,0.0,1010,54,56,63,50,7.8,3,0.0,0.002979,-0.050847,-0.128205,0.0
185,70,88,0.3,1016,51,52,54,50,6.6,3,0.0,0.005941,-0.071429,0.294118,0.0
186,70,73,0.2,1008,53,54,60,50,7.8,3,0.0,-0.007874,0.038462,-0.170455,0.0


In [16]:
# Assign X (data) and y (target)
X = hist_ml_df.drop("Migraine", axis=1)
y = hist_ml_df["Migraine"]
print(X.shape, y.shape)

(187, 14) (187,)


In [17]:
#Split our data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# MinMax Scaler

In [18]:
#Fit on the training data
X_scaler = MinMaxScaler().fit(X_train)
X_scaler.get_params()

{'copy': True, 'feature_range': (0, 1)}

In [19]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_test_scaled

array([[ 7.00000000e-01,  6.41791045e-01,  8.33333333e-02,
         2.05882353e-01,  6.72413793e-01,  6.08695652e-01,
         6.53061224e-01,  6.36363636e-01,  5.30864198e-01,
         4.00000000e-01,  0.00000000e+00,  2.25177589e-01,
         5.34412955e-01,  3.07497735e-01],
       [ 8.00000000e-01,  7.46268657e-01,  3.33333333e-01,
         8.23529412e-01,  2.93103448e-01,  2.17391304e-01,
         1.63265306e-01,  2.04545455e-01,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  6.52070972e-01,
         1.86602871e-01,  4.80735212e-01],
       [ 7.00000000e-02,  5.52238806e-01,  0.00000000e+00,
         5.29411765e-01,  4.65517241e-01,  4.13043478e-01,
         4.89795918e-01,  4.09090909e-01,  6.41975309e-01,
         4.00000000e-01,  0.00000000e+00,  6.54033338e-01,
         7.16599190e-01,  4.73958600e-01],
       [ 8.60000000e-01,  8.80597015e-01,  4.16666667e-02,
         6.17647059e-01,  4.82758621e-01,  4.34782609e-01,
         4.89795918e-01,  3.86363636e-01,  2.

In [20]:
y_train.value_counts()

0.0    88
1.0    52
Name: Migraine, dtype: int64

In [21]:
y_test.value_counts()

0.0    33
1.0    14
Name: Migraine, dtype: int64

In [22]:
model = SVC(kernel="linear")
model

SVC(kernel='linear')

# Predictions Before Grid Search

In [23]:
model.fit(X_train, y_train)

SVC(kernel='linear')

In [24]:
print('Test Acc: %.3f' % model.score(X_test_scaled, y_test))

Test Acc: 0.298


In [25]:
# Calculate classification report
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
print(classification_report(y_test, predictions,
                            target_names=["0","1"]))

              precision    recall  f1-score   support

           0       0.76      0.76      0.76        33
           1       0.43      0.43      0.43        14

    accuracy                           0.66        47
   macro avg       0.59      0.59      0.59        47
weighted avg       0.66      0.66      0.66        47



In [26]:
print('Test Acc: %.3f' % model.score(X_test_scaled, y_test))

Test Acc: 0.298


In [27]:
# Calculate classification report
from sklearn.metrics import classification_report
predictions = model.predict(X_test_scaled)
print(classification_report(y_test, predictions,
                            target_names=["0","1"]))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        33
           1       0.30      1.00      0.46        14

    accuracy                           0.30        47
   macro avg       0.15      0.50      0.23        47
weighted avg       0.09      0.30      0.14        47



C:\Users\yenia\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Predictions Using Grid Search

In [28]:
# Create the GridSearch estimator along with a parameter object containing the values to adjust
param_grid = {'C': [1, 5, 10, 50],
              'gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [29]:
# Fit the model using the grid search estimator. 
# This will take the SVC model and try each combination of parameters
grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.607, total=   0.0s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.607, total=   0.0s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.679, total=   0.0s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.750, total=   0.0s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.714, total=   0.0s
[CV] C=1, gamma=0.0005 ...............................................
[CV] ................... C=1, gamma=0.0005, score=0.607, total=   0.0s
[CV] C=1, gamma=0.0005 ...............................................
[CV] ...........

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV] .................... C=1, gamma=0.001, score=0.714, total=   0.0s
[CV] C=1, gamma=0.005 ................................................
[CV] .................... C=1, gamma=0.005, score=0.607, total=   0.0s
[CV] C=1, gamma=0.005 ................................................
[CV] .................... C=1, gamma=0.005, score=0.607, total=   0.0s
[CV] C=1, gamma=0.005 ................................................
[CV] .................... C=1, gamma=0.005, score=0.679, total=   0.0s
[CV] C=1, gamma=0.005 ................................................
[CV] .................... C=1, gamma=0.005, score=0.750, total=   0.0s
[CV] C=1, gamma=0.005 ................................................
[CV] .................... C=1, gamma=0.005, score=0.714, total=   0.0s
[CV] C=5, gamma=0.0001 ...............................................
[CV] ................... C=5, gamma=0.0001, score=0.571, total=   0.0s
[CV] C=5, gamma=0.0001 ...............................................
[CV] .

[CV] ................... C=50, gamma=0.001, score=0.607, total=   0.0s
[CV] C=50, gamma=0.001 ...............................................
[CV] ................... C=50, gamma=0.001, score=0.607, total=   0.0s
[CV] C=50, gamma=0.001 ...............................................
[CV] ................... C=50, gamma=0.001, score=0.714, total=   0.0s
[CV] C=50, gamma=0.005 ...............................................
[CV] ................... C=50, gamma=0.005, score=0.571, total=   0.0s
[CV] C=50, gamma=0.005 ...............................................
[CV] ................... C=50, gamma=0.005, score=0.500, total=   0.0s
[CV] C=50, gamma=0.005 ...............................................
[CV] ................... C=50, gamma=0.005, score=0.607, total=   0.0s
[CV] C=50, gamma=0.005 ...............................................
[CV] ................... C=50, gamma=0.005, score=0.607, total=   0.0s
[CV] C=50, gamma=0.005 ...............................................
[CV] .

[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    0.5s finished


GridSearchCV(estimator=SVC(kernel='linear'),
             param_grid={'C': [1, 5, 10, 50],
                         'gamma': [0.0001, 0.0005, 0.001, 0.005]},
             verbose=3)

In [30]:
# List the best parameters for this dataset
print(grid.best_params_)

{'C': 1, 'gamma': 0.0001}


In [31]:
# List the best score
print(grid.best_score_)

0.6714285714285715


In [32]:
# Make predictions with the hypertuned model
predictions = grid.predict(X_test_scaled)

In [33]:
print('Test Acc: %.3f' % grid.score(X_test_scaled, y_test))

Test Acc: 0.660


In [34]:
predictions

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1.])

In [35]:
# Calculate classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions,
                             target_names=["0","1"]))

              precision    recall  f1-score   support

           0       0.70      0.91      0.79        33
           1       0.25      0.07      0.11        14

    accuracy                           0.66        47
   macro avg       0.47      0.49      0.45        47
weighted avg       0.56      0.66      0.59        47



In [36]:
# history_df.to_csv("../../Desktop/test.csv")